In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive

/content/drive/MyDrive


In [3]:
!pip install transformers==2.8.0

     |████████████████████████████████| 563 kB 4.1 MB/s 
     |████████████████████████████████| 5.6 MB 35.9 MB/s 
     |████████████████████████████████| 131 kB 47.6 MB/s 
     |████████████████████████████████| 1.2 MB 38.7 MB/s 
     |████████████████████████████████| 895 kB 43.0 MB/s 
     |████████████████████████████████| 7.8 MB 39.1 MB/s 
     |████████████████████████████████| 79 kB 7.9 MB/s 
     |████████████████████████████████| 138 kB 54.8 MB/s 
     |████████████████████████████████| 127 kB 52.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
!pip3 install "prompt_toolkit==2.0.1" -q
!pip3 install -U ipython -q

     |████████████████████████████████| 322 kB 3.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 2.0.1 which is incompatible.
ipython 5.5.0 requires prompt-toolkit<2.0.0,>=1.0.4, but you have prompt-toolkit 2.0.1 which is incompatible.
     |████████████████████████████████| 786 kB 4.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 2.0.1 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.26.0 which is incompatible.


In [5]:
!pip3 install rasa[convert]

     |████████████████████████████████| 766 kB 4.3 MB/s 
     |████████████████████████████████| 59 kB 7.0 MB/s 
     |████████████████████████████████| 1.6 MB 29.7 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 1.3 MB 29.4 MB/s 
     |████████████████████████████████| 46 kB 3.4 MB/s 
     |████████████████████████████████| 111 kB 44.6 MB/s 
     |████████████████████████████████| 113 kB 45.3 MB/s 
     |████████████████████████████████| 3.4 MB 32.3 MB/s 
     |████████████████████████████████| 722 kB 24.6 MB/s 
     |████████████████████████████████| 320.6 MB 55 kB/s 
     |████████████████████████████████| 246 kB 53.1 MB/s 
     |████████████████████████████████| 72 kB 590 kB/s 
     |████████████████████████████████| 96 kB 6.8 MB/s 
     |████████████████████████████████| 40 kB 6.1 MB/s 
     |████████████████████████████████| 42 kB 905 kB/s 
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |█████████████████████████

In [6]:
from rasa.cli.utils import get_validated_path
from rasa.model import get_model, get_model_subdirectories
from rasa.nlu.model import Interpreter
from sklearn.metrics import precision_recall_fscore_support

def evaluation(dataf):
  y_true = dataf['intent'].tolist()
  y_pred = dataf['pred_intent'].tolist()
  metrics = precision_recall_fscore_support(y_true, y_pred, average='macro',zero_division=0)
  return np.array((metrics[0:3]))


def load_interpreter(model_path):
    """
    This loads the Rasa NLU interpreter. It is able to apply all NLU
    pipeline steps to a text that you provide it. 
    """
    model = get_validated_path(model_path, "model")
    model_path = get_model(model)
    _, nlu_model = get_model_subdirectories(model_path)
    return Interpreter.load(nlu_model)


import rasa.shared.nlu.training_data.loading as nlu_loading
train_data = nlu_loading.load_data("intent-dispatcher/faq_chitchat_data/multi_class_data/1cv_test_unbiased.yml")
# This `train_data` object contains intent_examples. This is a 
# list of `Message` objects. These are containers that can 
# contain intents, entities but also other information that 
# is relevant to a NLU pipeline.

def data_load(path):
  raw_data = nlu_loading.load_data(path)
  data = [m.as_dict() for m in raw_data.intent_examples]
  return data

[m.as_dict() for m in train_data.intent_examples][0:5]

[{'intent': 'faq_fur', 'text': 'is the fur used in your products real?'},
 {'intent': 'faq_fur', 'text': 'are all ASOS products vegan?'},
 {'intent': 'faq_fur', 'text': 'do you use vegan fur?'},
 {'intent': 'faq_delivery_slot',
  'text': 'I need my parcel by the latest on Thursday'},
 {'intent': 'faq_delivery_slot',
  'text': 'I need my parcel by the latest on Friday'}]

In [ ]:
nlu_interpreter = load_interpreter("intent-dispatcher/faq_chitchat_data/multi_class_data/model_unbiased/cv1_DIET.tar.gz")
nlu_interpreter.pipeline 

/usr/local/lib/python3.7/dist-packages/rasa/utils/train_utils.py:646: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss. It will be set to `True` by default, Rasa Open Source 3.0.0 onwards.
  category=UserWarning,


In [ ]:
nlu_interpreter.parse("I need my parcel by the latest on Thursday")

In [10]:
import pandas as pd 
import numpy as np

def add_predictions(dataf, nlu):
    """This function will add prediction columns based on `nlu`"""
    pred_blob = [nlu.parse(t)['intent'] for t in dataf['text']]
    return (dataf
            [['text', 'intent']]
            .assign(pred_intent=[p['name'] for p in pred_blob])
            .assign(pred_confidence=[p['confidence'] for p in pred_blob]))


def two_step_add_predictions(dataf, nlu1, nlu2):
    "Two-step intent dispatcher, nul1 is binary classfier, nlu2 is faq multi-class classifier"
    pred_blob = [nlu1.parse(t)['intent'] for t in dataf['text']]

    binary_dataf=(dataf
              [['text', 'intent']]
              .assign(pred_intent=[p['name'] for p in pred_blob])
              .assign(pred_confidence=[p['confidence'] for p in pred_blob]))
    # print(binary_dataf)
    for i in range(binary_dataf.shape[0]):
      if binary_dataf.loc[i,'pred_intent']== 'faq':
        text = binary_dataf.loc[i,'text']
        binary_dataf.loc[i,'pred_intent'] = nlu2.parse(text)['intent']['name']

    return binary_dataf

# First make a list of dictionaries that contain the utterances. 
data_list = [m.as_dict() for m in train_data.intent_examples]
# Turn this list into a dataframe and add predictions using the nlu-interpreter 
# df_intents = pd.DataFrame(data_list).pipe(add_predictions, nlu=nlu_interpreter)
# df_intents = pd.DataFrame(data_list).pipe(two_step_add_predictions, nlu1=nlu_interpreter,nlu2=nlu_interpreter)
# df_intents

In [ ]:
  pre_int = two_step_add_predictions(dataf, nlu1, nlu2)

# Path of Models and Test Set

In [ ]:
!pwd

/content/drive/My Drive


In [8]:
test_path = ['intent-dispatcher/faq_chitchat_data/multi_class_data/1cv_test_unbiased.yml',
        'intent-dispatcher/faq_chitchat_data/multi_class_data/2cv_test_unbiased.yml',
        'intent-dispatcher/faq_chitchat_data/multi_class_data/3cv_test_unbiased.yml',
        'intent-dispatcher/faq_chitchat_data/multi_class_data/4cv_test_unbiased.yml',
        'intent-dispatcher/faq_chitchat_data/multi_class_data/5cv_test_unbiased.yml']

faq_test_path = ['FAQ/data/1cv_original_test.yml',
          'FAQ/data/2cv_original_test.yml',
          'FAQ/data/3cv_original_test.yml',
          'FAQ/data/4cv_original_test.yml',
          'FAQ/data/5cv_original_test.yml',]


# Pipeline 1: full multi-class dispatcher models


DIET_full = ['intent-dispatcher/faq_chitchat_data/multi_class_data/model_unbiased/cv1_DIET.tar.gz',
        'intent-dispatcher/faq_chitchat_data/multi_class_data/model_unbiased/cv2_DIET.tar.gz',
        'intent-dispatcher/faq_chitchat_data/multi_class_data/model_unbiased/cv3_DIET.tar.gz',
         'intent-dispatcher/faq_chitchat_data/multi_class_data/model_unbiased/cv4_DIET.tar.gz',
          'intent-dispatcher/faq_chitchat_data/multi_class_data/model_unbiased/cv5_DIET.tar.gz']

DIET_Pre_full = ['intent-dispatcher/faq_chitchat_data/multi_class_data/model_unbiased/cv1_DIET_Pretrained.tar.gz',
           'intent-dispatcher/faq_chitchat_data/multi_class_data/model_unbiased/cv2_DIET_Pretrained.tar.gz',
           'intent-dispatcher/faq_chitchat_data/multi_class_data/model_unbiased/cv3_DIET_Pretrained.tar.gz',
           'intent-dispatcher/faq_chitchat_data/multi_class_data/model_unbiased/cv4_DIET_Pretrained.tar.gz',
           'intent-dispatcher/faq_chitchat_data/multi_class_data/model_unbiased/cv5_DIET_Pretrained.tar.gz']

BERT_full = ['intent-dispatcher/faq_chitchat_data/multi_class_data/model_unbiased/cv1_BERT.tar.gz',
         'intent-dispatcher/faq_chitchat_data/multi_class_data/model_unbiased/cv2_BERT.tar.gz',
         'intent-dispatcher/faq_chitchat_data/multi_class_data/model_unbiased/cv3_BERT.tar.gz',
         'intent-dispatcher/faq_chitchat_data/multi_class_data/model_unbiased/cv4_BERT.tar.gz',
         'intent-dispatcher/faq_chitchat_data/multi_class_data/model_unbiased/cv5_BERT.tar.gz']


# ------------------------------------------------------------------------------------------------------------------------------
# Pipeline 2: binary intent dispatcher and faq multi-class classifier


DIET_binary = ['intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv1_DIET.tar.gz',
         'intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv2_DIET.tar.gz',
         'intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv3_DIET.tar.gz',
         'intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv4_DIET.tar.gz',
         'intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv5_DIET.tar.gz']

DIET_Pre_binary = ['intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv1_DIET_Pretrained.tar.gz',
           'intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv2_DIET_Pretrained.tar.gz',
           'intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv3_DIET_Pretrained.tar.gz',
           'intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv4_DIET_Pretrained.tar.gz',
           'intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv5_DIET_Pretrained.tar.gz']

BERT_binary = ['intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv1_BERT.tar.gz',
         'intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv2_BERT.tar.gz',
         'intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv3_BERT.tar.gz',
         'intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv4_BERT.tar.gz',
         'intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv5_BERT.tar.gz']


# multi-class dispatcher only for FAQ (with augmentation)
DIET_faq = ['FAQ/models/cv1_DIET.tar.gz',
           'FAQ/models/cv2_DIET.tar.gz',
           'FAQ/models/cv3_DIET.tar.gz',
           'FAQ/models/cv4_DIET.tar.gz',
           'FAQ/models/cv5_DIET.tar.gz']

DIET_Pre_faq = ['FAQ/models/cv1_DIET_Pretrained.tar.gz',
           'FAQ/models/cv2_DIET_Pretrained.tar.gz',
           'FAQ/models/cv3_DIET_Pretrained.tar.gz',
           'FAQ/models/cv4_DIET_Pretrained.tar.gz',
           'FAQ/models/cv5_DIET_Pretrained.tar.gz']

BERT_faq = ['FAQ/models/cv1_BERT.tar.gz',
           'FAQ/models/cv2_BERT.tar.gz',
           'FAQ/models/cv3_BERT.tar.gz',
           'FAQ/models/cv4_BERT.tar.gz',
           'FAQ/models/cv5_BERT.tar.gz']

# FAQ classifier without augmentation

DIET_original = ['FAQ/models_without_augment/cv1_DIET.tar.gz',
           'FAQ/models_without_augment/cv2_DIET.tar.gz',
           'FAQ/models_without_augment/cv3_DIET.tar.gz',
           'FAQ/models_without_augment/cv4_DIET.tar.gz',
           'FAQ/models_without_augment/cv5_DIET.tar.gz']

DIET_Pre_original = ['FAQ/models_without_augment/cv1_DIET_Pretrained.tar.gz',
           'FAQ/models_without_augment/cv2_DIET_Pretrained.tar.gz',
           'FAQ/models_without_augment/cv3_DIET_Pretrained.tar.gz',
           'FAQ/models_without_augment/cv4_DIET_Pretrained.tar.gz',
           'FAQ/models_without_augment/cv5_DIET_Pretrained.tar.gz']

BERT_original = ['FAQ/models_without_augment/cv1_BERT.tar.gz',
           'FAQ/models_without_augment/cv2_BERT.tar.gz',
           'FAQ/models_without_augment/cv3_BERT.tar.gz',
           'FAQ/models_without_augment/cv4_BERT.tar.gz',
           'FAQ/models_without_augment/cv5_BERT.tar.gz']


In [ ]:
!pwd

/content/drive/My Drive


In [ ]:
nlu = load_interpreter(BERT_binary[0])


# FAQ classification without Augmentation Data

In [ ]:
DIET_faq_metric = np.array([0,0,0])

for i in range(len(DIET_original)):
  test_data = data_load(faq_test_path[i])
  nlu = load_interpreter(DIET_original[i])
  pre_int = pd.DataFrame(test_data).pipe(add_predictions, nlu=nlu)
  metrics = evaluation(pre_int)
  print('fold_{}'.format(i+1))
  print('precision ','recall ','f1_score ')
  print(metrics)
  DIET_faq_metric = DIET_faq_metric+ metrics

DIET_faq_metric = DIET_faq_metric/5

print('average: ', DIET_faq_metric)

/usr/local/lib/python3.7/dist-packages/rasa/utils/train_utils.py:646: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss. It will be set to `True` by default, Rasa Open Source 3.0.0 onwards.
  category=UserWarning,


fold_1
precision  recall  f1_score 
[0.75       0.796875   0.74791667]


fold_2
precision  recall  f1_score 
[0.8125     0.828125   0.80520833]


fold_3
precision  recall  f1_score 
[0.7421875  0.72395833 0.7125    ]


fold_4
precision  recall  f1_score 
[0.72247024 0.71875    0.69097222]


fold_5
precision  recall  f1_score 
[0.77864583 0.78125    0.75729167]
average:  [0.76116071 0.76979167 0.74277778]


In [ ]:
DIET_Pre_faq_metric = np.array([0,0,0])

for i in range(len(DIET_Pre_original)):
  test_data = data_load(faq_test_path[i])
  nlu = load_interpreter(DIET_Pre_original[i])
  pre_int = pd.DataFrame(test_data).pipe(add_predictions, nlu=nlu)
  metrics = evaluation(pre_int)
  print('fold_{}'.format(i+1))
  print('precision ','recall ','f1_score ')
  print(metrics)
  DIET_Pre_faq_metric = DIET_Pre_faq_metric+ metrics

DIET_Pre_faq_metric = DIET_Pre_faq_metric/5

print('average: ', DIET_Pre_faq_metric)

/usr/local/lib/python3.7/dist-packages/rasa/utils/train_utils.py:646: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss. It will be set to `True` by default, Rasa Open Source 3.0.0 onwards.
  category=UserWarning,


fold_1
precision  recall  f1_score 
[0.875      0.83854167 0.83333333]


fold_2
precision  recall  f1_score 
[0.80729167 0.84375    0.815625  ]


fold_3
precision  recall  f1_score 
[0.77552083 0.78125    0.74880952]


fold_4
precision  recall  f1_score 
[0.83854167 0.84375    0.82708333]


fold_5
precision  recall  f1_score 
[0.79947917 0.796875   0.78020833]
average:  [0.81916667 0.82083333 0.8010119 ]


In [13]:
BERT_faq_metric = np.array([0,0,0])

for i in range(len(BERT_original)):
  test_data = data_load(faq_test_path[i])
  nlu = load_interpreter(BERT_original[i])
  pre_int = pd.DataFrame(test_data).pipe(add_predictions, nlu=nlu)
  metrics = evaluation(pre_int)
  print('fold_{}'.format(i+1))
  print('precision ','recall ','f1_score ')
  print(metrics)
  BERT_faq_metric = BERT_faq_metric + metrics

BERT_faq_metric = BERT_faq_metric/5

print('average: ', BERT_faq_metric)

/usr/local/lib/python3.7/dist-packages/rasa/utils/train_utils.py:646: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss. It will be set to `True` by default, Rasa Open Source 3.0.0 onwards.
  category=UserWarning,


fold_1
precision  recall  f1_score 
[0.83333333 0.79166667 0.78645833]
fold_2
precision  recall  f1_score 
[0.82291667 0.8125     0.796875  ]
fold_3
precision  recall  f1_score 
[0.76302083 0.77083333 0.746875  ]
fold_4
precision  recall  f1_score 
[0.78854167 0.78125    0.76056548]
fold_5
precision  recall  f1_score 
[0.78333333 0.796875   0.76889881]
average:  [0.79822917 0.790625   0.77193452]


# FAQ classification with Augmentation classification

In [14]:
DIET_metric_aug = np.array([0,0,0])

for i in range(len(DIET_faq)):
  test_data = data_load(faq_test_path[i])
  nlu = load_interpreter(DIET_faq[i])
  pre_int = pd.DataFrame(test_data).pipe(add_predictions, nlu=nlu)
  metrics = evaluation(pre_int)
  print('fold_{}'.format(i+1))
  print('precision ','recall ','f1_score ')
  print(metrics)
  DIET_metric_aug = DIET_metric_aug + metrics

DIET_metric_aug = DIET_metric_aug/5

print('average: ', DIET_metric_aug)

/usr/local/lib/python3.7/dist-packages/rasa/utils/train_utils.py:646: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss. It will be set to `True` by default, Rasa Open Source 3.0.0 onwards.
  category=UserWarning,


fold_1
precision  recall  f1_score 
[0.85104167 0.77604167 0.77098214]
fold_2
precision  recall  f1_score 
[0.8125     0.8125     0.78645833]
fold_3
precision  recall  f1_score 
[0.76041667 0.75520833 0.73229167]
fold_4
precision  recall  f1_score 
[0.76510417 0.75       0.73139881]
fold_5
precision  recall  f1_score 
[0.78333333 0.796875   0.76577381]
average:  [0.79447917 0.778125   0.75738095]


In [15]:
DIET_Pre_metric_aug = np.array([0,0,0])

for i in range(len(DIET_Pre_faq)):
  test_data = data_load(faq_test_path[i])
  nlu = load_interpreter(DIET_Pre_faq[i])
  pre_int = pd.DataFrame(test_data).pipe(add_predictions, nlu=nlu)
  metrics = evaluation(pre_int)
  print('fold_{}'.format(i+1))
  print('precision ','recall ','f1_score ')
  print(metrics)
  DIET_Pre_metric_aug = DIET_Pre_metric_aug + metrics

DIET_Pre_metric_aug = DIET_Pre_metric_aug/5

print('average: ', DIET_Pre_metric_aug)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://github.com/connorbrinton/polyai-models/releases/download/v1.0/model.tar.gz'.
INFO:absl:Downloaded https://github.com/connorbrinton/polyai-models/releases/download/v1.0/model.tar.gz, Total size: 152.02MB
INFO:absl:Downloaded TF-Hub Module 'https://github.com/connorbrinton/polyai-models/releases/download/v1.0/model.tar.gz'.


/usr/local/lib/python3.7/dist-packages/rasa/utils/train_utils.py:646: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss. It will be set to `True` by default, Rasa Open Source 3.0.0 onwards.
  category=UserWarning,


fold_1
precision  recall  f1_score 
[0.93489583 0.90104167 0.9       ]


fold_2
precision  recall  f1_score 
[0.87239583 0.828125   0.82470238]


fold_3
precision  recall  f1_score 
[0.84114583 0.859375   0.83511905]


fold_4
precision  recall  f1_score 
[0.88020833 0.84375    0.83645833]


fold_5
precision  recall  f1_score 
[0.91666667 0.875      0.87395833]
average:  [0.8890625  0.86145833 0.85404762]


In [16]:
BERT_metric_aug = np.array([0,0,0])

for i in range(len(BERT_faq)):
  test_data = data_load(faq_test_path[i])
  nlu = load_interpreter(BERT_faq[i])
  pre_int = pd.DataFrame(test_data).pipe(add_predictions, nlu=nlu)
  metrics = evaluation(pre_int)
  print('fold_{}'.format(i+1))
  print('precision ','recall ','f1_score ')
  print(metrics)
  BERT_metric_aug = BERT_metric_aug + metrics

BERT_metric_aug = BERT_metric_aug/5

print('average: ', BERT_metric_aug)

/usr/local/lib/python3.7/dist-packages/rasa/utils/train_utils.py:646: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss. It will be set to `True` by default, Rasa Open Source 3.0.0 onwards.
  category=UserWarning,


fold_1
precision  recall  f1_score 
[0.8828125  0.83854167 0.8375    ]


fold_2
precision  recall  f1_score 
[0.90364583 0.859375   0.85595238]


fold_3
precision  recall  f1_score 
[0.8125     0.81770833 0.80520833]


fold_4
precision  recall  f1_score 
[0.78645833 0.8125     0.78333333]


fold_5
precision  recall  f1_score 
[0.921875 0.890625 0.884375]
average:  [0.86145833 0.84375    0.83327381]


In [18]:
DIET_metric =  [0.76116071, 0.76979167, 0.74277778]
DIET_Pre_metric = [0.81916667, 0.82083333, 0.8010119 ]
BERT_metric = [0.79822917, 0.790625,   0.77193452]

In [19]:
# Pipeline 1:
comparision3 = pd.DataFrame(index=['F1', 'Precision', 'Recall'], columns=['CountVectorsFeat.', 'ConveRT Feat.', 'BERT Feat.'])
comparision3.loc['F1','CountVectorsFeat.']=DIET_metric[2]
comparision3.loc['F1','ConveRT Feat.']=DIET_Pre_metric[2]
comparision3.loc['F1','BERT Feat.']=BERT_metric[2]

comparision3.loc['Precision','CountVectorsFeat.']=DIET_metric[0]
comparision3.loc['Precision','ConveRT Feat.']=DIET_Pre_metric[0]
comparision3.loc['Precision','BERT Feat.']=BERT_metric[0]

comparision3.loc['Recall','CountVectorsFeat.']=DIET_metric[1]
comparision3.loc['Recall','ConveRT Feat.']=DIET_Pre_metric[1]
comparision3.loc['Recall','BERT Feat.']=BERT_metric[1]

In [20]:
# Pipeline 1:
comparision4 = pd.DataFrame(index=['F1', 'Precision', 'Recall'], columns=['CountVectorsFeat.', 'ConveRT Feat.', 'BERT Feat.'])
comparision4.loc['F1','CountVectorsFeat.']=DIET_metric_aug[2]
comparision4.loc['F1','ConveRT Feat.']=DIET_Pre_metric_aug[2]
comparision4.loc['F1','BERT Feat.']=BERT_metric_aug[2]

comparision4.loc['Precision','CountVectorsFeat.']=DIET_metric_aug[0]
comparision4.loc['Precision','ConveRT Feat.']=DIET_Pre_metric_aug[0]
comparision4.loc['Precision','BERT Feat.']=BERT_metric_aug[0]

comparision4.loc['Recall','CountVectorsFeat.']=DIET_metric_aug[1]
comparision4.loc['Recall','ConveRT Feat.']=DIET_Pre_metric_aug[1]
comparision4.loc['Recall','BERT Feat.']=BERT_metric_aug[1]

In [21]:
comparision3

,CountVectorsFeat.,ConveRT Feat.,BERT Feat.
F1,0.742778,0.801012,0.771935
Precision,0.761161,0.819167,0.798229
Recall,0.769792,0.820833,0.790625


In [23]:
print('with augmentation data')
comparision4

with augmentation data


,CountVectorsFeat.,ConveRT Feat.,BERT Feat.
F1,0.757381,0.854048,0.833274
Precision,0.794479,0.889062,0.861458
Recall,0.778125,0.861458,0.84375


# One-step Dispatching

In [ ]:
# DIET precision, recall, and precision
import numpy as np

DIET = np.array([0,0,0])

for i in range(len(DIET_full)):
  test_data = data_load(test_path[i])
  nlu = load_interpreter(DIET_full[i])
  pre_int = pd.DataFrame(test_data).pipe(add_predictions, nlu=nlu)
  metrics = evaluation(pre_int)
  print('fold_{}'.format(i+1))
  print('precision ','recall ','f1_score ')
  print(metrics)
  DIET = DIET+ metrics

DIET = DIET/5

print('average: ', DIET)


/usr/local/lib/python3.7/dist-packages/rasa/utils/train_utils.py:646: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss. It will be set to `True` by default, Rasa Open Source 3.0.0 onwards.
  category=UserWarning,


fold_1
precision  recall  f1_score 
[0.88372457 0.7979798  0.80711916]
fold_2
precision  recall  f1_score 
[0.82193362 0.78787879 0.78115461]
fold_3
precision  recall  f1_score 
[0.75836941 0.73187095 0.72212637]
fold_4
precision  recall  f1_score 
[0.78697422 0.78787879 0.77327824]
fold_5
precision  recall  f1_score 
[0.87538428 0.81818182 0.81874629]
average:  [0.82527722 0.78475803 0.78048493]


In [ ]:
# DIET with Pretrained

DIET_Pre = np.array([0,0,0])

for i in range(len(DIET_Pre_full)):
  test_data = data_load(test_path[i])
  nlu = load_interpreter(DIET_Pre_full[i])
  pre_int = pd.DataFrame(test_data).pipe(add_predictions, nlu=nlu)
  metrics = evaluation(pre_int)
  print('fold_{}'.format(i+1))
  print('precision ','recall ','f1_score ')
  print(metrics)
  DIET_Pre = DIET_Pre + metrics

DIET_Pre = DIET_Pre/5

print('average: ', DIET_Pre)

/usr/local/lib/python3.7/dist-packages/rasa/utils/train_utils.py:646: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss. It will be set to `True` by default, Rasa Open Source 3.0.0 onwards.
  category=UserWarning,


fold_1
precision  recall  f1_score 
[0.92171717 0.9040404  0.90505051]


fold_2
precision  recall  f1_score 
[0.90151515 0.87878788 0.86839827]


fold_3
precision  recall  f1_score 
[0.82950938 0.8030303  0.79803773]


fold_4
precision  recall  f1_score 
[0.88383838 0.84848485 0.84141414]


fold_5
precision  recall  f1_score 
[0.88383838 0.87878788 0.86565657]
average:  [0.88408369 0.86262626 0.85571144]


In [ ]:
# BERT with Pretrained

BERT = np.array([0,0,0])

for i in range(len(BERT_full)):
  test_data = data_load(test_path[i])
  nlu = load_interpreter(BERT_full[i])
  pre_int = pd.DataFrame(test_data).pipe(add_predictions, nlu=nlu)
  metrics = evaluation(pre_int)
  print('fold_{}'.format(i+1))
  print('precision ','recall ','f1_score ')
  print(metrics)
  BERT = BERT + metrics

BERT = BERT/5
print('average: ', BERT)

Downloading:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/277 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/654 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

...

fold_1
precision  recall  f1_score 
[0.90819964 0.84343434 0.85005277]


fold_2
precision  recall  f1_score 
[0.85814023 0.86363636 0.84320988]


fold_3
precision  recall  f1_score 
[0.82323232 0.83793155 0.82401458]


fold_4
precision  recall  f1_score 
[0.87832874 0.84848485 0.84118282]


fold_5
precision  recall  f1_score 
[0.86363636 0.87833559 0.85734792]
average:  [0.86630746 0.85436454 0.84316159]


In [ ]:
binary_dataf
for i in range(binary_dataf[0]):
if binary_dataf.loc[i,'pred_intent']== 'faq':
  text = binary_dataf.loc[i,'text']
  binary_dataf.loc[i,'pred_intent'] = nlu2.parse(text)['intent']['name']

# Two-Step Dispatching

In [ ]:
# Two-setp intent dispatcher

# DIET precision, recall, and precision

DIET_p2 = np.array([0,0,0])

for i in range(len(DIET_binary)):
  test_data = data_load(test_path[i])
  nlu1 = load_interpreter(DIET_binary[i])
  nlu2 = load_interpreter(DIET_faq[i])
  dataf = pd.DataFrame(test_data)
  #pre_int = two_step_add_predictions(dataf, nlu1, nlu2)
  
  pre_int = pd.DataFrame(test_data).pipe(two_step_add_predictions, nlu1=nlu1, nlu2=nlu2)
  metrics = evaluation(pre_int)
  print('fold_{}'.format(i+1))
  print('precision ','recall ','f1_score ')
  print(metrics)
  DIET_p2 = DIET_p2 + metrics

DIET_p2 = DIET_p2/5

print('average: ', DIET_p2)

/usr/local/lib/python3.7/dist-packages/rasa/utils/train_utils.py:646: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss. It will be set to `True` by default, Rasa Open Source 3.0.0 onwards.
  category=UserWarning,


fold_1
precision  recall  f1_score 
[0.86520428 0.78282828 0.78375484]


fold_2
precision  recall  f1_score 
[0.82480024 0.81772953 0.79565161]


fold_3
precision  recall  f1_score 
[0.76170799 0.76126941 0.73522442]


fold_4
precision  recall  f1_score 
[0.77939846 0.75757576 0.74600551]


fold_5
precision  recall  f1_score 
[0.7793457  0.80257802 0.76635868]
average:  [0.80209133 0.7843962  0.76539901]


In [ ]:
DIET_Pre_p2 = np.array([0,0,0])

for i in range(len(DIET_Pre_binary)):
  test_data = data_load(test_path[i])
  nlu1 = load_interpreter(DIET_Pre_binary[i])
  nlu2 = load_interpreter(DIET_Pre_faq[i])
  pre_int = pd.DataFrame(test_data).pipe(two_step_add_predictions, nlu1=nlu1, nlu2=nlu2)
  metrics = evaluation(pre_int)
  print('fold_{}'.format(i+1))
  print('precision ','recall ','f1_score ')
  print(metrics)
  DIET_Pre_p2 = DIET_Pre_p2 + metrics

DIET_Pre_p2 = DIET_Pre_p2/5

print('average: ', DIET_Pre_p2)

/usr/local/lib/python3.7/dist-packages/rasa/utils/train_utils.py:646: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss. It will be set to `True` by default, Rasa Open Source 3.0.0 onwards.
  category=UserWarning,


fold_1
precision  recall  f1_score 
[0.93686869 0.9040404  0.9030303 ]


fold_2
precision  recall  f1_score 
[0.84551396 0.81818182 0.80958794]


fold_3
precision  recall  f1_score 
[0.84551396 0.84848485 0.82978996]


fold_4
precision  recall  f1_score 
[0.82232776 0.81818182 0.80055096]


fold_5
precision  recall  f1_score 
[0.91919192 0.87878788 0.87777778]
average:  [0.87388326 0.85353535 0.84414739]


In [ ]:
print(BERT_binary[0])
nlu1 = load_interpreter(BERT_binary[0])

In [ ]:
import numpy as np
BERT_p2 = np.array([0,0,0])
print(BERT_binary)

for i in range(len(DIET_Pre_binary)):
  print(i)
  test_data = data_load(test_path[i])
  nlu1 = load_interpreter(BERT_binary[i])
  nlu2 = load_interpreter(BERT_faq[i])

  nlu1 = load_interpreter(BERT_binary[i])
  nlu2 = load_interpreter(BERT_faq[i])
  pre_int = pd.DataFrame(test_data).pipe(two_step_add_predictions, nlu1=nlu1, nlu2=nlu2)
  metrics = evaluation(pre_int)
  print('fold_{}'.format(i+1))
  print('precision ','recall ','f1_score ')
  print(metrics)
  BERT_p2 = BERT_p2 + metrics

BERT_p2 = BERT_p2/5
print('average: ', BERT_p2)

['intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv1_BERT.tar.gz', 'intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv2_BERT.tar.gz', 'intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv3_BERT.tar.gz', 'intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv4_BERT.tar.gz', 'intent-dispatcher/faq_chitchat_data/binary_data/unbiased_data/model_unbiased/cv5_BERT.tar.gz']
0


Downloading:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/277 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/654 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/rasa/utils/train_utils.py:646: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss. It will be set to `True` by default, Rasa Open Source 3.0.0 onwards.
  category=UserWarning,


fold_1
precision  recall  f1_score 
[0.89304813 0.82828283 0.83490125]
1
fold_2
precision  recall  f1_score 
[0.87538428 0.83333333 0.8295688 ]
2
fold_3
precision  recall  f1_score 
[0.79247016 0.80717624 0.78921546]
3
fold_4
precision  recall  f1_score 
[0.79247016 0.8030303  0.77956666]
4
fold_5
precision  recall  f1_score 
[0.9338978  0.89393939 0.89371493]
average:  [0.8574541  0.83315242 0.82539342]


In [ ]:
DIET = [0.82527722, 0.78475803, 0.78048493]
DIET_Pre = [0.88408369, 0.86262626, 0.85571144]
BERT = [0.86630746, 0.85436454, 0.84316159]

DIET_p2 = [0.80209133, 0.7843962,  0.76539901]
DIET_Pre_p2 = [0.87388326, 0.85353535, 0.84414739]
BERT_p2 = [0.8574541, 0.83315242, 0.82539342]

In [ ]:
# Pipeline 1:
comparision = pd.DataFrame(index=['F1', 'Precision', 'Recall'], columns=['CountVectorsFeat.', 'ConveRT Feat.', 'BERT Feat.'])
comparision.loc['F1','CountVectorsFeat.']=DIET[2]
comparision.loc['F1','ConveRT Feat.']=DIET_Pre[2]
comparision.loc['F1','BERT Feat.']=BERT[2]

comparision.loc['Precision','CountVectorsFeat.']=DIET[0]
comparision.loc['Precision','ConveRT Feat.']=DIET_Pre[0]
comparision.loc['Precision','BERT Feat.']=BERT[0]

comparision.loc['Recall','CountVectorsFeat.']=DIET[1]
comparision.loc['Recall','ConveRT Feat.']=DIET_Pre[1]
comparision.loc['Recall','BERT Feat.']=BERT[1]

comparision

,CountVectorsFeat.,ConveRT Feat.,BERT Feat.
F1,0.780485,0.855711,0.843162
Precision,0.825277,0.884084,0.866307
Recall,0.784758,0.862626,0.854365


In [ ]:
# Pipeline 2: two_step dispatching 
comparision2 = pd.DataFrame(index=['F1', 'Precision', 'Recall'], columns=['CountVectorsFeat.', 'ConveRT Feat.', 'BERT Feat.'])
comparision2.loc['F1','CountVectorsFeat.']=DIET_p2[2]
comparision2.loc['F1','ConveRT Feat.']=DIET_Pre_p2[2]
comparision2.loc['F1','BERT Feat.']=BERT_p2[2]
comparision2.loc['Precision','CountVectorsFeat.']=DIET_p2[0]
comparision2.loc['Precision','ConveRT Feat.']=DIET_Pre_p2[0]
comparision2.loc['Precision','BERT Feat.']=BERT_p2[0]
comparision2.loc['Recall','CountVectorsFeat.']=DIET_p2[1]
comparision2.loc['Recall','ConveRT Feat.']=DIET_Pre_p2[1]
comparision2.loc['Recall','BERT Feat.']=BERT_p2[1]
print('two_step dispatching')
comparision2

two_step dispatching


,CountVectorsFeat.,ConveRT Feat.,BERT Feat.
F1,0.765399,0.844147,0.825393
Precision,0.802091,0.873883,0.857454
Recall,0.784396,0.853535,0.833152


In [ ]:
from sklearn.metrics import precision_recall_fscore_support

def evaluation(dataf):
  y_true = dataf['intent'].tolist()
  y_pred = dataf['pred_intent'].tolist()
  metrics = precision_recall_fscore_support(y_true, y_pred, average='macro',zero_division=0)
  return np.array((metrics[0:3]))

# pre_int

In [ ]:
pre_int

y_true = pre_int['intent'].tolist()
y_pred = pre_int['pred_intent'].tolist()
metrics = precision_recall_fscore_support(y_true, y_pred, average='macro',zero_division=0)


[(0.8837245696400625, 0.7979797979797979, 0.8071191582140488, None)]

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score


y_true = df_intents['intent'].tolist()
y_pred = df_intents['pred_intent'].tolist()

f1= f1_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
precision = precision_score(y_true, y_pred, average='macro')

print(precision,recall,f1)

0.8837245696400625 0.7979797979797979 0.8071191582140488


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import precision_recall_fscore_support



# y_true = df_intents['intent'].tolist()
# y_pred = df_intents['pred_intent'].tolist()
# metrics = precision_recall_fscore_support(y_true, y_pred, average='macro',zero_division=0)

def evaluation(dataf):
  y_true = dataf['intent'].tolist()
  y_pred = dataf['pred_intent'].tolist()
  metrics = precision_recall_fscore_support(y_true, y_pred, average='macro',zero_division=0)
  return np.array((metrics[0:3]))